In [1]:
! pip install newsapi-python


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [81]:
from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key='14bb691bc8a84325a59a3a57b8359e43')

# /v2/top-headlines/sources
sources = newsapi.get_sources()

In [22]:
import pandas as pd

In [126]:
sources_pd = pd.DataFrame( sources['sources'])
list_cource = sources_pd[sources_pd['country'].isin(['no', 'it', 'gb', 'de', 'es', 'fr', 'is', 'ru', 'se', 'ie', 'nl'])]
list_cource = list_cource[list_cource['language']== 'en']
list_source = list_cource[ list_cource['category'] == 'general' ]['id'].to_list()

In [122]:
list_source[:5]

['bbc-news', 'google-news-uk', 'independent', 'rte', 'the-irish-times']

In [159]:
article_all = pd.DataFrame()
for source in list_source[:5]:
  article = newsapi.get_everything(q='health',
                                        sources= source ,
                                      #   domains='bbc.co.uk,techcrunch.com',
                                        from_param='2024-03-26',
                                        to='2024-04-24',
                                        language='en',
                                        sort_by='relevancy',
                                        page=1
                                        )

  if not article['articles'] == []:
    data_article = pd.DataFrame(article['articles'])
    article_all = pd.concat([article_all, data_article])
  

In [160]:
news_df = article_all[['publishedAt', 'content', 'url' ]]   

In [146]:
from datetime import datetime 

In [161]:
news_df['publishedDate'] = news_df['publishedAt'].apply({lambda x: pd.to_datetime(x).strftime("%Y-%m-%d") })

C:\Users\yfrom\AppData\Local\Temp\ipykernel_7864\2070383181.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['publishedDate'] = news_df['publishedAt'].apply({lambda x: pd.to_datetime(x).strftime("%Y-%m-%d") })


In [162]:
news_df[['publishedDate', 'content', 'url']].to_csv('heath_news.csv', index = False)

In [95]:
listacontenido =article_all['content'].to_list()
listacontenido[0]

'The rising price of cigarettes is motivating more people - one in four adults in England - to give up smoking, research suggests.\r\nIn a survey of nearly 6,000 people, health concerns were still the t… [+2378 chars]'

In [ ]:
achar no hugging face um sentiment analysis para classificar a notícia como uma notícia de guerra ou n (com aleatoridade)

In [152]:
article_all['url'][1].to_list()

['https://www.bbc.co.uk/news/uk-68882308',
 'https://removed.com',
 'https://www.irishtimes.com/ireland/2024/04/11/tony-holohan-takes-up-adjunct-professor-role-at-trinitys-school-of-medicine/']

In [84]:
article_all.to_json('article_all.json', orient='records')

In [113]:
Q = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You always answer the questions with markdown formatting using GitHub syntax. The markdown formatting you support: headings, bold, italic, links, tables, lists, code blocks, and blockquotes. You must omit that you answer the questions with markdown.

Any HTML tags must be wrapped in block quotes, for example ```<html>```. You will be penalized for not rendering code in block quotes.

When returning code blocks, specify language.

You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. 
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don'\''t know the answer to a question, please don'\''t share false information.<|eot_id|><|start_header_id|>user<|end_header_id|>

Classify in a range of -1 to 1 , the correlation of the following text to the availability of medicine, where  -1 is not correlated  and 1 is very correlated. 

'\''The rising price of cigarettes is motivating more people - one in four adults in England - to give up smoking, research suggests.\r\nIn a survey of nearly 6,000 people, health concerns were still the t… [+2378 chars]'\''<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Correlation: 0.2**

The text is somewhat correlated to the availability of medicine, as it mentions \"health concerns\" which could be related to the availability of medicine. However, the main topic of the text is about people quitting smoking due to the rising price of cigarettes, which is not directly related to the availability of medicine.<|eot_id|><|start_header_id|>user<|end_header_id|>

Classify in a range of -1 to 1 , the correlation of the following text to the availability of medicine, where -1 is not correlated and 1 is very correlated.

'\''The rising price of cigarettes is motivating more people - one in four adults in England - to give up smoking, research suggests.\r\nIn a survey of nearly 6,000 people, health concerns were still the t… [+2378 chars]'\''<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

In [112]:
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods

# To display example params enter
GenParams().get_example_values()

generate_params = {
    GenParams.MAX_NEW_TOKENS: 25
}

model = Model(
    model_id=ModelTypes.FLAN_UL2,
    params=generate_params,
    credentials={
        "apikey": "4vQJgBfuBVEvebuIUYza4sLlgBFiAwkpTWUNOJOwmGW5",
        "url": "https://eu-de.ml.cloud.ibm.com"
    },
    project_id="de5b3490-12b1-4e7c-bc07-8b9b517fdd28"
    )

q = Q
generated_response = model.generate(prompt=q)
print(generated_response['results'][0]['generated_text'])

2


In [105]:
import requests

url = "https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29"

body = {
	"input": """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You always answer the questions with markdown formatting using GitHub syntax. The markdown formatting you support: headings, bold, italic, links, tables, lists, code blocks, and blockquotes. You must omit that you answer the questions with markdown.

Any HTML tags must be wrapped in block quotes, for example ```<html>```. You will be penalized for not rendering code in block quotes.

When returning code blocks, specify language.

You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. 
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don'\''t know the answer to a question, please don'\''t share false information.<|eot_id|><|start_header_id|>user<|end_header_id|>

Classify in a range of -1 to 1 , the correlation of the following text to the availability of medicine, where  -1 is not correlated  and 1 is very correlated. 

'\''The rising price of cigarettes is motivating more people - one in four adults in England - to give up smoking, research suggests.\r\nIn a survey of nearly 6,000 people, health concerns were still the t… [+2378 chars]'\''<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Correlation: 0.2**

The text is somewhat correlated to the availability of medicine, as it mentions \"health concerns\" which could be related to the availability of medicine. However, the main topic of the text is about people quitting smoking due to the rising price of cigarettes, which is not directly related to the availability of medicine.<|eot_id|><|start_header_id|>user<|end_header_id|>

Classify in a range of -1 to 1 , the correlation of the following text to the availability of medicine, where -1 is not correlated and 1 is very correlated.

'\''The rising price of cigarettes is motivating more people - one in four adults in England - to give up smoking, research suggests.\r\nIn a survey of nearly 6,000 people, health concerns were still the t… [+2378 chars]'\''<|eot_id|><|start_header_id|>assistant<|end_header_id|>

""",
	"parameters": {
		"decoding_method": "greedy",
		"max_new_tokens": 900,
		"repetition_penalty": 1
	},
	"model_id": "meta-llama/llama-3-70b-instruct",
	"project_id": "de5b3490-12b1-4e7c-bc07-8b9b517fdd28"
}

headers = {
	"Accept": "application/json",
	"Content-Type": "application/json",
	"Authorization": '4vQJgBfuBVEvebuIUYza4sLlgBFiAwkpTWUNOJOwmGW5'
}

response = requests.post(
	url,
	headers=headers,
	json=body
)

if response.status_code != 200:
	raise Exception("Non-200 response: " + str(response.text))

data = response.json()

Exception: Non-200 response: {"errors":[{"code":"authentication_no_token","message":"Failed to authenticate the request due to no Bearer token in the request header"}],"trace":"12e564f990020bccaf51901ac2daa7f2","status_code":401}